<a href="https://colab.research.google.com/github/shahmehul2005/RIS-placement-using-CNN/blob/main/mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Libraries**


In [22]:
!pip install pandas geopy opencv-python ultralytics matplotlib folium

### **Mount Google Drive**


In [23]:
import google.colab
google.colab.drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Upload Metadata File**


In [24]:
from google.colab import files
uploaded = files.upload()


Saving flight_metadata (6).csv to flight_metadata (6) (1).csv


## **Imports and Dependencies**

In [25]:
import cv2
import pandas as pd
import numpy as np
import math
import torch
import os
import folium
from geopy.distance import geodesic
from ultralytics import YOLO
import matplotlib.pyplot as plt

print("Libraries imported successfully.")

Libraries imported successfully.


## **Configuration Parameters**

In [26]:
# --- File Paths ---
METADATA_FILE = 'flight_metadata (6).csv'
IMAGE_DIRECTORY = '/content/drive/MyDrive/drone_images_lnm2'

# --- Model Paths ---
# Ensure these point to your specific trained weights
LOW_ALT_MODEL_PATH = '/content/drive/MyDrive/Copy of high.pt' # YOLOv5
HIGH_ALT_MODEL_PATH = '/content/drive/MyDrive/Copy of best.pt'  # YOLOv8

# --- Mapping Parameters ---
MAP_ZOOM_LEVEL = 20
GSD_METERS_PER_PIXEL = 0.262 # Baseline GSD

# --- Algorithm Parameters ---
# The number of overlapping images that must detect an object
# for it to be considered valid (Noise reduction).
DETECTION_CONFIDENCE_THRESHOLD = 2

# --- Visualization Colors ---
OBSTACLE_COLORS_WEB = {
    'building': 'red',
    'tree': 'darkgreen',
    'low vegetation': 'lightgreen',
    'transmission tower': 'orange',
    'communication tower': 'blue',
    'tower': 'purple'
}
DEFAULT_COLOR_WEB = 'white'

## **Obstacle Detector Class Definition**

In [27]:
class ObstacleDetector:
    """
    Wrapper class to handle switching between Low Altitude (YOLOv5)
    and High Altitude (YOLOv8) models transparently.
    """
    def __init__(self, low_alt_model_path, high_alt_model_path):
        self.low_alt_yolov5_path = low_alt_model_path
        self.high_alt_yolov8_path = high_alt_model_path

        # Class definitions based on model training
        self.low_alt_classes = {48: 'building', 59: 'tower'}
        self.high_alt_classes = ['building', 'tree', 'low vegetation',
                                 'transmission tower', 'communication tower']

        self.low_alt_model = None
        self.high_alt_model = None

    def load_low_alt_model(self):
        """Lazy loader for YOLOv5"""
        if self.low_alt_model is None:
            self.low_alt_model = torch.hub.load('ultralytics/yolov5', 'custom',
                                                path=self.low_alt_yolov5_path,
                                                force_reload=True, trust_repo=True)
        return self.low_alt_model

    def load_high_alt_model(self):
        """Lazy loader for YOLOv8"""
        if self.high_alt_model is None:
            self.high_alt_model = YOLO(self.high_alt_yolov8_path)
        return self.high_alt_model

    def postprocess_yolov8(self, results):
        """Standardize YOLOv8 output to list of dicts"""
        detections = []
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    # Safe class name retrieval
                    if class_id < len(self.high_alt_classes):
                        class_name = self.high_alt_classes[class_id]
                    else:
                        class_name = f'class_{class_id}'

                    detections.append({
                        'bbox': [int(x1), int(y1), int(x2), int(y2)],
                        'class_name': class_name
                    })
        return detections

    def postprocess_yolov5(self, results):
        """Standardize YOLOv5 output to list of dicts"""
        detections = []
        results_df = results.pandas().xyxy[0]
        for _, row in results_df.iterrows():
            class_id = int(row['class'])
            if class_id in self.low_alt_classes:
                detections.append({
                    'bbox': [int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])],
                    'class_name': self.low_alt_classes[class_id]
                })
        return detections

    def detect_obstacles(self, image, height):
        """
        Main inference method. Selects model based on drone altitude.
        """
        if height > 100:
            model = self.load_low_alt_model()
            results = model(image)
            return self.postprocess_yolov5(results)
        else:
            model = self.load_high_alt_model()
            results = model(image, conf=0.25, verbose=False)
            return self.postprocess_yolov8(results)

## **Geospatial Helper Functions**

In [28]:
def calculate_gsd(latitude, zoom_level):
    """
    Calculates Ground Sample Distance (meters/pixel) dynamically
    based on latitude and zoom level.
    """
    return (156543.03 * math.cos(latitude * math.pi / 180)) / (2 ** zoom_level)

def calculate_canvas_size(df, gsd_avg):
    """
    Determines the dimensions of the virtual canvas needed to cover
    the entire flight area.
    """
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

    # Origin is the Top-Left corner (Max Lat, Min Lon)
    top_left = (max_lat, min_lon)

    map_height_meters = geodesic(top_left, (min_lat, min_lon)).meters
    map_width_meters = geodesic(top_left, (max_lat, max_lon)).meters

    canvas_height_px = math.ceil(map_height_meters / gsd_avg)
    canvas_width_px = math.ceil(map_width_meters / gsd_avg)

    return canvas_width_px, canvas_height_px, top_left

def pixel_to_gps(origin_lat, origin_lon, x_pixels, y_pixels, gsd):
    """
    Converts a specific pixel coordinate on the virtual canvas back
    to a real-world GPS coordinate.
    """
    distance_meters_x = x_pixels * gsd
    distance_meters_y = y_pixels * gsd

    # Calculate point moving East, then South
    temp_point = geodesic(meters=distance_meters_x).destination((origin_lat, origin_lon), bearing=90)
    final_point = geodesic(meters=distance_meters_y).destination(temp_point, bearing=180)

    return (final_point.latitude, final_point.longitude)

## **Initialization**

In [29]:
# 1. Load Metadata
try:
    metadata_df = pd.read_csv(METADATA_FILE)
    print(f"Loaded metadata for {len(metadata_df)} images.")
except FileNotFoundError:
    raise FileNotFoundError(f"Metadata file not found at '{METADATA_FILE}'")

if not os.path.isdir(IMAGE_DIRECTORY):
    raise FileNotFoundError(f"Image directory not found at '{IMAGE_DIRECTORY}'")

# 2. Calculate Map Center and Canvas
map_center_coords = [metadata_df['latitude'].mean(), metadata_df['longitude'].mean()]
avg_gsd = calculate_gsd(map_center_coords[0], MAP_ZOOM_LEVEL)

canvas_width, canvas_height, map_origin_coords = calculate_canvas_size(metadata_df, avg_gsd)
print(f"Virtual Canvas Size: {canvas_width}px wide x {canvas_height}px high")

# 3. Initialize Confidence Grids (The Voting System)
# We create one zero-filled grid for each obstacle class
confidence_grids = {
    class_name: np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    for class_name in OBSTACLE_COLORS_WEB.keys()
}

# 4. Initialize Folium Map
m = folium.Map(location=map_center_coords, zoom_start=18,
               tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google')

# 5. Initialize Detector
try:
    detector = ObstacleDetector(LOW_ALT_MODEL_PATH, HIGH_ALT_MODEL_PATH)
    print("ObstacleDetector initialized successfully.")
except Exception as e:
    print(f"Error initializing detector: {e}")

Loaded metadata for 289 images.
Virtual Canvas Size: 4776px wide x 5328px high
ObstacleDetector initialized successfully.


## **Processing Images and Updating Grids**

In [30]:
print(f"--- Starting Phase 1: Processing {len(metadata_df)} Images ---")

for index, row in metadata_df.iterrows():
    # 1. Load Image Data
    image_filename = row['filename']
    image_lat, image_lon, altitude = row['latitude'], row['longitude'], row['altitude']
    image_path = os.path.join(IMAGE_DIRECTORY, image_filename)

    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load image '{image_filename}'. Skipping.")
        continue

    h, w, _ = image.shape
    print(f"Processing {image_filename} ({index + 1}/{len(metadata_df)})", end='\r')

    # 2. Run Detection
    detections = detector.detect_obstacles(image, altitude)

    # 3. Calculate Coordinates (Center-Corrected)
    gsd = calculate_gsd(image_lat, MAP_ZOOM_LEVEL)

    dy_meters_center = geodesic(map_origin_coords, (image_lat, map_origin_coords[1])).meters
    dx_meters_center = geodesic(map_origin_coords, (map_origin_coords[0], image_lon)).meters

    center_x_px = int(dx_meters_center / gsd)
    center_y_px = int(dy_meters_center / gsd)

    # Shift to Top-Left anchor
    x_offset = center_x_px - (w // 2)
    y_offset = center_y_px - (h // 2)

    # 4. Vote on Confidence Grids
    for detection in detections:
        class_name = detection['class_name']

        if class_name in confidence_grids:
            x1, y1, x2, y2 = detection['bbox']

            # Calculate global coordinates on canvas
            global_tl_px_x = x_offset + x1
            global_tl_px_y = y_offset + y1
            global_br_px_x = x_offset + x2
            global_br_px_y = y_offset + y2

            # Get grid and define Region of Interest (ROI)
            grid = confidence_grids[class_name]

            # Handle edge cases (clipping to canvas boundaries)
            y1_b, y2_b = max(0, global_tl_px_y), min(grid.shape[0], global_br_px_y)
            x1_b, x2_b = max(0, global_tl_px_x), min(grid.shape[1], global_br_px_x)

            # Increment Vote (Saturated Addition)
            if y1_b < y2_b and x1_b < x2_b:
                roi = grid[y1_b:y2_b, x1_b:x2_b]
                grid[y1_b:y2_b, x1_b:x2_b] = np.clip(roi.astype(np.uint16) + 1, 0, 255).astype(np.uint8)

print("\nPhase 1 Complete. Grids populated.")

--- Starting Phase 1: Processing 289 Images ---
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


YOLOv5 🚀 2025-11-19 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 322 layers, 86570425 parameters, 0 gradients, 205.0 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processing drone_frame_0288.png (289/289)
Phase 1 Complete. Grids populated.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


## **Merging, Contouring, and Map Generation**

In [31]:
print(f"--- Starting Phase 2: Merging Detections and Plotting ---")

detected_classes_for_legend = set()

for class_name, grid in confidence_grids.items():
    # 1. Apply Confidence Threshold
    # Only keep pixels detected in at least 'DETECTION_CONFIDENCE_THRESHOLD' images
    _, mask = cv2.threshold(grid, DETECTION_CONFIDENCE_THRESHOLD - 1, 255, cv2.THRESH_BINARY)

    # 2. Find Contours (Shapes) of the merged obstacles
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        continue

    print(f"Found {len(contours)} merged obstacles for class: '{class_name}'")
    detected_classes_for_legend.add(class_name)

    # 3. Convert Contours to GPS Polygons
    for contour in contours:
        # Simplify contour to reduce file size
        epsilon = 0.001 * cv2.arcLength(contour, True)
        approx_contour = cv2.approxPolyDP(contour, epsilon, True)

        gps_polygon = []
        for point in approx_contour:
            x, y = point[0]
            gps_coord = pixel_to_gps(map_origin_coords[0], map_origin_coords[1], x, y, avg_gsd)
            gps_polygon.append(gps_coord)

        # Filter out noise (tiny polygons)
        if len(gps_polygon) < 3:
            continue

        # 4. Add to Folium Map
        color = OBSTACLE_COLORS_WEB.get(class_name, DEFAULT_COLOR_WEB)

        folium.Polygon(
            locations=gps_polygon,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"<b>Obstacle:</b> {class_name.title()}"
        ).add_to(m)

# --- Add Legend ---
legend_html = '''
<div style="position: fixed;
            top: 50px; right: 50px; width: 200px; height: auto;
            background-color: rgba(255, 255, 255, 0.9); border:2px solid grey; z-index:9999;
            font-family: Arial, sans-serif; font-size: 14px;
            padding: 10px; border-radius: 5px;">
<b>Legend</b><br>
'''

for class_name in detected_classes_for_legend:
    color = OBSTACLE_COLORS_WEB.get(class_name, DEFAULT_COLOR_WEB)
    legend_html += f'<i style="background:{color}; border:1px solid #777; width:18px; height:18px; float:left; margin-right:8px; opacity: 0.7;"></i> {class_name.title()}<br>'

if not detected_classes_for_legend:
    legend_html += "No obstacles detected."

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# --- Save Output ---
output_filename = 'final_interactive_merged_map.html'
m.save(output_filename)

print(f"\n--- All Done ---")
print(f"Final MERGED map saved as '{output_filename}'.")

--- Starting Phase 2: Merging Detections and Plotting ---
Found 50 merged obstacles for class: 'building'

--- All Done ---
Final MERGED map saved as 'final_interactive_merged_map.html'.
